In [1]:
import numpy as np
from enum import Enum
import random 
import copy
import pickle 
from tqdm import tqdm_notebook
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, RNN, Conv2D, MaxPooling2D, Flatten, Dropout
import time, datetime
from IPython.display import clear_output
from keras.models import model_from_json
from uttt import UltimateTicTacToe
from helper import Turns, GameState, switch_turns, current_milli_time
import os
import keras

Using TensorFlow backend.


In [57]:
input_shape = (18,9,1)
num_output_neurons = 2

In [120]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(18,9,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))


model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [59]:
def save(model):
    timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
    # saving the model
    model_json = model.to_json()
    filename = "models/"+timestamp
    os.makedirs(filename, exist_ok=True)
    with open("models/"+timestamp+"/model.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("models/"+timestamp+"/weights.h5")
    print("Saved model to disk as with "+timestamp+" timestamp")

In [60]:
initial_board = np.array([np.zeros(9) for _ in range(9)])
UTTT = UltimateTicTacToe(board=initial_board,last_turn=None)

In [75]:
def play_a_random_game(): # returns the memory of the game with the winner
    memory = []
    uttt = copy.deepcopy(UTTT)
    simulation_turn = Turns.X.value
    while not uttt.is_game_done():
        old_board = uttt.get_board_list().flatten()
        move = random.choice(uttt.get_free_moves())
        uttt.move(simulation_turn,*move)
        new_board = uttt.get_board_list().flatten()
        state = np.concatenate((old_board,new_board),axis=0) 
        simulation_turn = switch_turns(simulation_turn)
        memory.append(state.reshape(18,9,1))
    winner = uttt.get_winner() if uttt.get_winner() != None else 0
    return memory, winner

In [97]:
def feature_and_label_of_memory(memory_len, winner, q_val=0.6, stretch=1):
    labels = []
    for i in range(memory_len):
        label = None
        value = q_val**(memory_len-(i+1)) * stretch
        if winner==Turns.X.value:
            label = np.array([value,0])
        elif winner==Turns.O.value:
            label = np.array([0,value])
        else:
            half_prob = value
            label = np.array([half_prob, half_prob])
        labels.append(label)
    return labels

In [122]:
BATCH_SIZE = 1000
features, labels = [], []
for _ in tqdm_notebook(range(BATCH_SIZE),desc="Batch Run..."):
    feature, winner = play_a_random_game()
    label = feature_and_label_of_memory(len(feature), winner)
    features.extend(feature)
    labels.extend(label)
features = np.array(features)
labels = np.array(labels)

In [123]:
model.fit(features, labels, epochs=10, batch_size=100)

Epoch 1/10
58286/58286 [==============================] - 3s 49us/step - loss: 0.0372 - acc: 0.6203
Epoch 2/10
58286/58286 [==============================] - 3s 48us/step - loss: 0.0340 - acc: 0.6293
Epoch 3/10
58286/58286 [==============================] - 3s 56us/step - loss: 0.0287 - acc: 0.6451
Epoch 4/10
58286/58286 [==============================] - 3s 47us/step - loss: 0.0229 - acc: 0.6651
Epoch 5/10
58286/58286 [==============================] - 3s 49us/step - loss: 0.0200 - acc: 0.6757
Epoch 6/10
58286/58286 [==============================] - 3s 44us/step - loss: 0.0189 - acc: 0.6741
Epoch 7/10
58286/58286 [==============================] - 3s 53us/step - loss: 0.0183 - acc: 0.6813
Epoch 8/10
58286/58286 [==============================] - 3s 48us/step - loss: 0.0180 - acc: 0.6782
Epoch 9/10
58286/58286 [==============================] - 3s 49us/step - loss: 0.0177 - acc: 0.6806
Epoch 10/10
58286/58286 [==============================] - 3s 45us/step - loss: 0.0175 - acc: 0.6845

In [124]:
save(model)

Saved model to disk as with 2018-07-30 19:39:39 timestamp
